In [63]:
#http://drivendata.co/blog/worldbank-poverty-benchmark/

%matplotlib inline

import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# data directory
DATA_DIR = os.path.join(r'C:\Users\piush\Desktop\Dataset\world_bank_poverty_data\household')

In [64]:
data_paths = {'A': {'train': os.path.join(DATA_DIR,  'A_hhold_train.csv'), 
                    'test':  os.path.join(DATA_DIR,  'A_hhold_test.csv')}, 
              
              'B': {'train': os.path.join(DATA_DIR,  'B_hhold_train.csv'), 
                    'test':  os.path.join(DATA_DIR,  'B_hhold_test.csv')}, 
              
              'C': {'train': os.path.join(DATA_DIR,  'C_hhold_train.csv'), 
                    'test':  os.path.join(DATA_DIR,  'C_hhold_test.csv')}}

In [65]:
# load training data
a_train = pd.read_csv(data_paths['A']['train'], index_col='id')
b_train = pd.read_csv(data_paths['B']['train'], index_col='id')
c_train = pd.read_csv(data_paths['C']['train'], index_col='id')

In [66]:
# load test data
a_test = pd.read_csv(data_paths['A']['test'], index_col='id')
b_test = pd.read_csv(data_paths['B']['test'], index_col='id')
c_test = pd.read_csv(data_paths['C']['test'], index_col='id')

In [67]:
b_train.shape

(3255, 442)

In [68]:
b_test.shape

(1604, 441)

In [69]:
df = pd.concat([b_train,b_test])

In [70]:
class LabelCount(object):

    def __init__(self, columns, new_column=False):
        self.count_dict = {}
        self.columns = columns
        self.new_column = new_column
        
    
    def fit(self, df):

        for column in self.columns:
            count = df[column].value_counts()
            
            self.count_dict[column] = count.to_dict()
        
    def transform(self, df):
        for column in self.columns:
            
            new_column_name = column
            
            if self.new_column:
                new_column_name = column + "_label_count"

            missing = 1
            df[new_column_name] = df[column].apply(lambda x : self.count_dict[column].get(x, missing))            


###### Label Count


In [71]:
for i in list(df.select_dtypes(include=['object']).columns.values):
    lc = LabelCount([i])
    lc.fit(df)
    lc.transform(df)
    

In [72]:
y = b_train['poor']

In [73]:
df = df.drop(['poor','country'], axis = 1)

#### Find columns with nans

In [74]:
df.columns[df.isnull().any()].tolist()

['BRzuVmyf',
 'BXOWgPgL',
 'FGWqGkmD',
 'IrxBnWxE',
 'McFBIGsm',
 'OSmfjCbE',
 'aAufyreG',
 'dnlnKrAg',
 'umkFMfvA']

###### Fill nan with median values

In [75]:
df['BRzuVmyf'].fillna(-1,inplace = True)

In [76]:
df['BXOWgPgL'].fillna(-1,inplace = True)

In [77]:
df['FGWqGkmD'].fillna(-1,inplace = True)

In [78]:
df['McFBIGsm'].fillna(-1,inplace = True)

In [79]:
df['OSmfjCbE'].fillna(-1,inplace = True)

In [80]:
df['IrxBnWxE'].fillna(-1,inplace = True)

In [81]:
df['aAufyreG'].fillna(-1,inplace = True)

In [82]:
df['dnlnKrAg'].fillna(-1,inplace = True)

In [83]:
df['umkFMfvA'].fillna(-1,inplace = True)

In [84]:
train = df[:len(b_train)]

In [85]:
test = df[len(b_train):]

##### Standard Scalar processing

In [86]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
scaler = RobustScaler()
scaler.fit(df)

train_data_scaled = scaler.transform(train)
test_data_scaled = scaler.transform(test)

In [121]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
class Ensemble(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models

    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)

        folds = list(StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=2016).split(X, y))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):

            S_test_i = np.zeros((T.shape[0], self.n_splits))

            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
#                y_holdout = y[test_idx]

                print ("Fit %s fold %d" % (str(clf).split('(')[0], j+1))
                clf.fit(X_train, y_train)
#                cross_score = cross_val_score(clf, X_train, y_train, cv=3, scoring='roc_auc')
#                print("    cross_score: %.5f" % (cross_score.mean()))
                y_pred = clf.predict_proba(X_holdout)[:,1]                

                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict_proba(T)[:,1]
            S_test[:, i] = S_test_i.mean(axis=1)

        results = cross_val_score(self.stacker, S_train, y, cv=3, scoring='neg_log_loss')
        print("Stacker score: %.5f" % (results.mean()))

        self.stacker.fit(S_train, y)
        res = self.stacker.predict_proba(S_test)[:,1]
        return res

In [87]:
from sklearn import model_selection

#from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from xgboost import XGBClassifier


test_size = 0.33

seed = 7

X_train, X_test, Y_train, Y_test = model_selection.train_test_split(train_data_scaled, y, test_size=test_size, random_state=seed)

#model = LogisticRegression()
#model = RandomForestClassifier(n_estimators=500, random_state=0)
# model = XGBClassifier(
#  learning_rate =0.1,
#  n_estimators=500,
#  max_depth=2,
#  min_child_weight=1,
#  gamma=0,
#  subsample=0.8,
#  colsample_bytree=0.8,
#  objective= 'binary:logistic',
#  nthread=4,
#  scale_pos_weight=1,
#  seed=27)

#model = XGBClassifier(max_depth=1, n_estimators=300, learning_rate=0.05, objective= "binary:logistic",min_child_weight = 1,scale_pos_weight = 1)
model = XGBClassifier(max_depth=3, learning_rate=0.06, n_estimators=500, 
                      silent=True, objective='binary:logistic',  
                      gamma=0.1, min_child_weight=1, max_delta_step=0.5, 
                      subsample=1, colsample_bytree=1, colsample_bylevel=1, 
                      reg_alpha=0, reg_lambda=1, scale_pos_weight= 1, 
                      base_score=0.5, seed= 100)

#model = SVC(gamma=2, C=1,probability= True)

model.fit(X_train, Y_train)

y_pred = model.predict_proba(X_test)

#print("f1_score: {}".format(f1_score(, average='micro')))
from sklearn.metrics import log_loss

log_loss(Y_test, y_pred)

0.27223719850615707

In [26]:
# #k-fold 

# num_instances = len(train_data_scaled)
# seed = 7

# kfold = model_selection.KFold(n_splits=5, random_state=seed)
# #model = LogisticRegression()
# model = XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05, objective= "binary:logistic",min_child_weight = 1,scale_pos_weight = 1)


# model_selection.cross_val_score(model, train_data_scaled, y, cv=kfold, scoring='neg_log_loss').mean()

-0.21139168561643315

In [88]:
 model.fit(train_data_scaled,y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0.1, learning_rate=0.06, max_delta_step=0.5, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=500, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=100, silent=True, subsample=1)

In [101]:

for _ in range(2):
    model.fit(train_data_scaled,y)
    a.append(model.predict_proba(test_data_scaled))
    

In [109]:
pd.Series(a)

0    [[0.975334, 0.0246657], [0.950807, 0.0491934],...
1    [[0.975334, 0.0246657], [0.950807, 0.0491934],...
dtype: object

In [100]:
# model.fit(train_data_scaled,y)
# model.predcit_proba(test_data_scaled)
   

In [119]:
model.fit(train_data_scaled,y)
b_preds = model.predict_proba(test_data_scaled)

In [120]:
b_preds

array([[ 0.97533429,  0.02466572],
       [ 0.95080662,  0.0491934 ],
       [ 0.99083531,  0.0091647 ],
       ..., 
       [ 0.99858671,  0.00141328],
       [ 0.95962787,  0.04037213],
       [ 0.94652444,  0.05347558]], dtype=float32)

In [118]:
b_preds

array([[ 0.97533429,  0.02466572],
       [ 0.95080662,  0.0491934 ],
       [ 0.99083531,  0.0091647 ],
       ..., 
       [ 0.99858671,  0.00141328],
       [ 0.95962787,  0.04037213],
       [ 0.94652444,  0.05347558]], dtype=float32)

In [110]:
def make_country_sub(preds, test_feat, country):
    # make sure we code the country correctly
    country_codes = ['A', 'B', 'C']
    
    # get just the poor probabilities
    country_sub = pd.DataFrame(data=preds[:, 1],  # proba p=1
                               columns=['poor'], 
                               index=test_feat.index)

    
    # add the country code for joining later
    country_sub["country"] = country
    return country_sub[["country", "poor"]]


In [111]:
# convert preds to data frames
b_sub = make_country_sub(b_preds, b_test, 'B')

In [112]:
#b_sub

In [113]:
b_sub.to_csv("b_sub_6.csv")